In [5]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Conv2D, MaxPool2D, Dropout, Embedding, GlobalAveragePooling1D, Activation
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy, BinaryCrossentropy

import os
import re
import shutil
import string
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
print('tensorflow version is {}'.format(tf.__version__))

In [ ]:
file='stack_overflow_16k'
url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'

In [ ]:
# # 뭔가 이상함, cache_subdir=file로 한 후 '' 해야 directory 구조가 원하는 대로 됨
dataset = tf.keras.utils.get_file(file, origin=url, untar=True, cache_dir='.' , cache_subdir=file)

In [ ]:
# # 뭔가 이상함, cache_subdir=file로 한 후 '' 해야 directory 구조가 원하는 대로 됨
dataset = tf.keras.utils.get_file(file, origin=url, untar=True, cache_dir='.' , cache_subdir='')

In [ ]:
os.listdir(dataset)

In [ ]:
train_dir = os.path.join(dataset, 'train')
train_dir_list = os.listdir(train_dir)

In [ ]:
train_dir_list

In [ ]:
train_dir_list = [i for i in train_dir_list if '.' != i[0]]

In [ ]:
train_dir_list

In [ ]:
indexs = np.random.randint(0, 2000, len(train_dir_list))

for idx, kind in enumerate(train_dir_list):
    file = os.listdir(os.path.join(train_dir, kind))[indexs[idx]]
    file = os.path.join(os.path.join(train_dir, kind), file)
    with open(file) as f:
        print(f.read())
    print('=========================================================')
    

In [ ]:
batch_size = 32
seed = 42

In [ ]:
raw_train = tf.keras.utils.text_dataset_from_directory(
    train_dir, batch_size=batch_size, validation_split=0.2, subset='training', seed=seed)

In [ ]:
a = next(iter(raw_train.take(1)))
print(a[0][0])
print(raw_train.class_names[a[1][0]])
print(a[1][0])

In [ ]:
for text, label in raw_train.take(1):
    for i in range(1):
        print('=================================')
        print(raw_train.class_names[label[i]])
        print('=================================')
        print(text[i].numpy())
        print('=================================')
        print(text[i])
        print('=================================')

In [ ]:
val_train = tf.keras.utils.text_dataset_from_directory(
    'stack_overflow_16k/train',
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

In [ ]:
test_dir = os.path.join(dataset + '/test')
test_dir

In [ ]:
test = tf.keras.utils.text_dataset_from_directory(test_dir, batch_size=batch_size)

In [ ]:
sub_dir = os.listdir(train_dir)[1]
sample_dir = os.path.join(train_dir, sub_dir)
file = os.listdir(sample_dir)[1]

In [ ]:
sample_file = os.path.join(sample_dir, file)
sample_file

In [ ]:
print(tf.strings.lower(sample_file))

In [ ]:
print(tf.strings.lower(sample_file).numpy())

In [ ]:
with open(sample_file) as f:
    sample_file = f.read()
    print(sample_file)

In [ ]:
lowercase = tf.strings.lower(sample_file)
print(lowercase.numpy())

In [ ]:
stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
print(stripped_html.numpy())

In [ ]:
stripped_html2 = tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')
print(stripped_html2.numpy())

In [ ]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label